In [9]:
import pickle
import pandas as pd
import re
import random

In [2]:
amazon = pd.read_excel("Amazon_Unmapped4.xlsx", header=0).values

# Divides mapped from unmapped
testing_data = amazon[amazon[:,3] != "Unmapped"]
amazon_unmapped = amazon[amazon[:,3] == "Unmapped"]

trainingDict = pickle.load(open("TrainingData.p","rb"))

In [3]:
# Divides description of each product into individual words
def splitter(df):
    split = [0]*df.shape[0]
    # Useless strings to be removed
    purge_list = ["","a","and","with","for","in","the","of","to","x","-","&"]
    
    # Divides description into individual strings
    for i in range(0,df.shape[0]):
        split[i] = [x.lower() for x in (re.split('[ _,+-\'/#?:;()|]',str(df[i][0])))]
        
        for j in purge_list:
            while(j in split[i]): 
                split[i].remove(j)
    return split

In [4]:
testing_corpus = splitter(testing_data)

In [5]:
# prop_tree = {
#     "Marvel": ["Avengers", "Marvel Other","Marvel Universe"]
#     "Avengers": ["Spider-Man""spider-man","iron man","guardian of the galaxy","guardians of the galaxy","captain america movie 3","thor movie 3",
#      "avengers movie 3 part 1","captain america movie - do not use","Spider-Man Movie (2017)","Ultimate Spider-Man","Thor - Do Not Use"]
# }

property_tree = [
    ["avengers","marvel","marvel universe","marvel other","spider-man","iron man","guardian of the galaxy","guardians of the galaxy","captain america movie 3","thor movie 3",
     "avengers movie 3 part 1","captain america movie - do not use","Spider-Man Movie (2017)","Ultimate Spider-Man","Thor - Do Not Use"],
    ["disney princess","frozen","disney frozen","frozen 2","frozen ii","beauty and the beast","little mermaid","cinderella","tangled","sleeping beauty","snow white & the 7 dwarfs",
     "Princess and the Frog","aladdin","Pocahontas","Mulan"],
    ["cars the movie","cars","cars 2","cars 3"],
    ["mickey mouse","mickey & friends","minnie","mickey mouse club"],
    ["star wars","star wars: rogue one","star wars - other","the mandalorian","star wars: the force awakens (episode 7)","The Phantom Menace (Episode 1)","The Empire Strikes Back (Episode 5)",
    "Star Wars Sequels - Other","Clone Wars"],
    ["frozen","disney frozen","frozen 2","frozen ii"],
    ["winnie the pooh","winnie the pooh & friends"],
    ["disney fairies","fairies"],
    ["finding nemo/finding dory","finding dory","finding nemo"],
    ["lilo & stitch","lilo and stitch"],
    ["nightmare before christmas","nightmare before xmas"],
    ["big hero 6","big hero six"],
    ["Lion Guard","The Lion Guard"],
    ["Monsters,  Inc.","Monsters Inc"],
    ["Moana/Vaiana","Moana"]
]
property_tree = [[x.lower() for x in sublist] for sublist in property_tree]

In [6]:
def testRandom(prodSplit, propList, thres, shouldMult):
    correct = 0
    total = 0
#     print("Running test...")
    ignored = 0
    for i in range(500):
        isCorrect = False
        index = random.randrange(len(propList))
        product = prodSplit[index]
#         print(product)
        top10 = {}
        top102 = {}
        followCheck = ""
        previousWord = ""
        for word in product:
#             print("Word: ", word)
            for property in trainingDict:
                for keyword in trainingDict[property]:
                    if word == keyword:
                        if property in top10:
                            if previousWord in trainingDict[property]:
                                if shouldMult: top10[property] += (trainingDict[property][keyword]*trainingDict[property][previousWord])-trainingDict[property][previousWord]
                                else: top10[property] *= trainingDict[property][keyword]
                            else:
                                top10[property] += trainingDict[property][keyword]
                        else:
                            top10[property] = trainingDict[property][keyword]

                        if property in top102:
                            top102[property] += 1
                        else:
                            top102[property] = 1
#                         print("Property: ", property, trainingDict[property][keyword])
            previousWord = word


        top10 = {k: v for k, v in sorted(top10.items(), key=lambda item: item[1], reverse=True)}
        top102 = {k: v for k, v in sorted(top102.items(), key=lambda item: item[1], reverse=True)}
        # ~40% of products are ignored
        if len(top10) < 2 or top10[next(iter(top10))]/top10[list(top10.keys())[1]] <= thres:
            ignored += 1
            continue
#         print(top10)
        total += 1
        if top10 != {}:
            if next(iter(top10)).lower() == propList[index].lower():
                correct += 1
                isCorrect = True
#                 print("Correct:", propList[index])
            else:
                for branch in property_tree:
                    if next(iter(top10)).lower() in branch and propList[index].lower() in branch:
                        correct += 1
                        isCorrect = True
#                         print("Correct:", next(iter(top10)), "&", propList[index])
                        break
                if not isCorrect:
                    print(product)
                    print("Incorrect:",next(iter(top10)),"\nAnswer:", propList[index], "\n")
#                     print("Correct? (t/f/na):")
#                     x = input()
#                     x = "f"
#                     if x == "t":
#                         correct += 1
#                         isCorrect = True
#                     elif x == "na":
#                         total -= 1
#                     else:
#                         print(top10, "\n")
                
    return ignored, correct, total

In [12]:
def predict(prodSplit):
    done = 0
    count = 0
    load = 500
    prev = 0
    
    propList = [0]*500
    print("Running prediction...")
    for i in range(500):
        index = random.randrange(len(prodSplit))
        product = prodSplit[index]
        top10 = {}
        top102 = {}
        followCheck = ""
        previousWord = ""
        for word in product:
            for property in trainingDict:
                for keyword in trainingDict[property]:
                    if word == keyword:
                        if property in top10:
                            if previousWord in trainingDict[property]:
                                # TODO: Must change so doesnt multiply into total property score but into previous word's score
                                top10[property] *= trainingDict[property][keyword]
                            else:
                                top10[property] += trainingDict[property][keyword]
                        else:
                            top10[property] = trainingDict[property][keyword]

                        if property in top102:
                            top102[property] += 1
                        else:
                            top102[property] = 1
            previousWord = word


        top10 = {k: v for k, v in sorted(top10.items(), key=lambda item: item[1], reverse=True)}
        top102 = {k: v for k, v in sorted(top102.items(), key=lambda item: item[1], reverse=True)}
        if len(top10) < 2 or top10[next(iter(top10))]/top10[list(top10.keys())[1]] <= 20: # This number will have to vary depending on size of data set
            propList[i] = [prodSplit[index],"Unmapped"]
            done += 1
            continue
        if top10 != {}:
            propList[i] = [prodSplit[index], next(iter(top10))]
        else:
            propList[i] = [prodSplit[index],"Unmapped"]
            done += 1
    print("Mapping complete.")
    print((load-done)*100/load, "% mapped")            
    return propList

In [18]:
# for i in [True, False]:
#     for j in range(5,105,5):
#         print(i,",", j,",", testRandom(testing_corpus,testing_data[:,3], j/10, i))
print(testRandom(testing_corpus,testing_data[:,3], 10, True))

['tiaobug', 'toddler', 'kids', 'little', 'girls', 'mermaid', 'costume', 'halloween', 'carnival', 'tutu', 't']
Incorrect: My Little Pony 
Answer: Little Mermaid 

['boys', 'kids', 'toy', 'story', 'cars', 'spiderman', 'superman', 'batman', 'star', 'wars', 'avengers', 'paw', 'p']
Incorrect: Star Wars 
Answer: Cars 

['johnny', 'lightning', 'marvel', 'spider-man', '1', '98', 'vw', 'volkswagon', 'beetle', 'mcfarla']
Incorrect: Spider-Man 
Answer: Cars 

['zootopia', 'mug']
Incorrect: Zootropolis 
Answer: Zootopia 

['girls', 'little', 'mermaid', 'long', 'sleeve', 'top', 'green', 'age', '8', '9', 'years']
Incorrect: My Little Pony 
Answer: Little Mermaid 

['peter', 'pan', 'tinkerbell', 'book', 'page', 'phone', 'case', 'fit', 'apple', 'iphone', 'all', 'mode']
Incorrect: Disney Heroes 
Answer: Fairies 

['fabric', '100%', 'cotton', 'characters', 'princess', 'winnie', 'toy', 'story', 'heros', '140cm']
Incorrect: Toy Story 
Answer: Winnie the Pooh 

['dc', 'comics', 'boy', 's', 'spiderman', 't-

In [14]:
x = predict(testing_corpus)

Running prediction...
Mapping complete.
61.6 % mapped


In [17]:
for i in range(20):
    j = random.randrange(len(x))
    if x[j][1] == "Unmapped": continue
    print(x[j],"\n")

[['pixar', 'toy', 'story', 'inspired', 'quote', 'pu', 'faux', 'leather', 'mouse', 'mat', 'pc', 'mouse', 'pa'], 'Toy Story'] 

[['phone', 'case', 'iphone', '8', 'venom', 'spider', 'man', 'eddie', 'brock', 'mac', 'gargan', 'marvel', 'comi'], 'Spider-Man'] 

[['marvel', 'spider-man', 'far', 'from', 'home', 'mysterio', 'tarot', 'card', 't-shirt'], 'Spider-Man'] 

[['marvel', 'toys', 'avengers', '3', '4-', '7-inch', 'iron', 'man', 'anti-hook', 'armor', 'joint', 'movable'], 'Avengers'] 

[['star', 'wars', 'tasse', 'rogue', 'one', 'stormtrooper', 'invasion', 'von', 'elbenwald'], 'Star Wars'] 

[['princess', 'belle', 'off', 'shoulder', 'layered', 'costume', 'dress', 'fancy', 'party', 'dress', 'lit'], 'Disney Princess'] 

[['aladdin', 'genie', 'princess', 'jasmine', 'walt', 'magic', 'carpet', 'magnetic', 'clasp', 'hold'], 'Disney Princess'] 

[['frozen', '2', 'clear', 'dome', 'umbrella'], 'Disney Frozen'] 

[['rapunzel', 'finds', 'friend', 'princess', 'random', 'house', 'hardcover'], 'Disney Pr